<a href="https://colab.research.google.com/github/Eliezer19Garcia/Machine_Learning/blob/main/ML_S8(Bayesian_Class_Practice).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Los clasificadores Naive Bayes (Naive Bayes Classifier - NBC) se usan , como su nombre indica, para problemas de clasificación, y en concreto se pueden aplicar para texto.

En este ejemplo vamos a implementar un modelo NBC a un dataset de un portal de Noticias muy famoso en España. Cada usuario comparte un link a una noticia y le puede asignar una categoría.

In [ ]:
import pandas as pd

noticias = pd.read_csv('noticias.csv')

print(noticias.shape)

noticias = noticias.sample(frac=0.60)
print(noticias.shape)


(16495, 2)
(9897, 2)


La variable objetivo es categoria y la variable independiente es descripcion que contiene la descripcion de la noticia

In [ ]:
noticias.categoria.value_counts()

,count
categoria,
cultura,5406
tecnología,2536
ocio,1955


Tenemos 3 tipos de categorias diferentes

Los clasificadores Naive Bayes esperan como input un vector, así que para poder entrenarlos tenemos que vectorizar el texto. Para ello una buena opción es usar vectorización Tf-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


Vamos a modificar nuestro vectorizador añadiendole dos parámetros:

** Eliminar Stopwords**

Stopwords (palabras vacías) son palabras que no tienen ningún contenido semántico. Por ejemplo, en la frase el perro ladra el artículo el no aporta ningún valor a la frase.

Me he descargado una lista de stopwords de castellano de este repositorio en Github.

In [ ]:
import json

with open("stopwords-es.json") as fname:
    stopwords_es = json.load(fname)

In [ ]:
stopwords_es[:50]

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '_',
 'a',
 'actualmente',
 'acuerdo',
 'adelante',
 'ademas',
 'además',
 'adrede',
 'afirmó',
 'agregó',
 'ahi',
 'ahora',
 'ahí',
 'al',
 'algo',
 'alguna',
 'algunas',
 'alguno',
 'algunos',
 'algún',
 'alli',
 'allí',
 'alrededor',
 'ambos',
 'ampleamos',
 'antano',
 'antaño',
 'ante',
 'anterior',
 'antes',
 'apenas',
 'aproximadamente',
 'aquel',
 'aquella',
 'aquellas',
 'aquello',
 'aquellos',
 'aqui',
 'aquél',
 'aquélla']

**Eliminar acentos**

También vamos a eliminar los acentos de las palabras, ésto tiene una ventaja, y es que si en el conjunto de datos no tenemos confianza en la calidad de los escritores, al eliminar los acentos evitamos que si un escritor no usa acentos no considere sus palabras como palabras distintas.

En castellano, esto tiene un problema, y es que hay palabras con significado distinto que sólo se diferencian por la existencia de una tilde (se llaman palabras con acento diacrítico (por ejemplo, `de` y `dé`). Asumimos pues que el impacto de estas palabras no es muy alto.

In [ ]:
vectorizador = TfidfVectorizer(strip_accents='unicode', stop_words=stopwords_es)

In [ ]:
noticias.shape

(9897, 2)

In [ ]:
vectorizador.fit_transform(noticias.descripcion)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afirmo', 'agrego', 'algun', 'anadio', 'aseguro', 'comento', 'considero', 'dejo', 'demas', 'estabamos', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'explico', 'expreso', 'fueramos', 'fuesemos', 'habeis', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'indico', 'llego', 'manifesto', 'menciono', 'ningun', 'podra', 'podran', 'proximos', 'quedo', 'realizo', 'seais', 'senalo', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 'tendria', 't

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 239561 stored elements and shape (9897, 46038)>

This matiz tell us tahht we have diferents articles rows with 59952 different words

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

Dado que los vectorizadores devuelven una matriz sparse (escasa) creamos un transformador que las convierta a densas

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer

from scipy.sparse import issparse


# http://rasbt.github.io/mlxtend/
class DenseTransformer(BaseEstimator):
    def __init__(self, return_copy=True):
        self.return_copy = return_copy
        self.is_fitted = False

    def transform(self, X, y=None):
        if issparse(X):
            return X.toarray()
        elif self.return_copy:
            return X.copy()
        else:
            return X

    def fit(self, X, y=None):
        self.is_fitted = True
        return self

    def fit_transform(self, X, y=None):
        return self.transform(X=X, y=y)


In [ ]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB

pipeline_gaussiano = make_pipeline(
    vectorizador,
    DenseTransformer(),
    GaussianNB()
)
#

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(noticias.descripcion, noticias.categoria, test_size=0.3)

In [ ]:
pipeline_gaussiano.fit(X=X_train, y=y_train)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(stop_words=['0', '1', '2', '3', '4', '5', '6',
                                             '7', '8', '9', '_', 'a',
                                             'actualmente', 'acuerdo',
                                             'adelante', 'ademas', 'además',
                                             'adrede', 'afirmó', 'agregó',
                                             'ahi', 'ahora', 'ahí', 'al',
                                             'algo', 'alguna', 'algunas',
                                             'alguno', 'algunos', 'algún', ...],
                                 strip_accents='unicode')),
                ('densetransformer', DenseTransformer()),
                ('gaussiannb', GaussianNB())])

In [ ]:
pipeline_gaussiano.predict(noticias.descripcion)

array(['cultura', 'cultura', 'cultura', ..., 'cultura', 'cultura',
       'cultura'], dtype='<U10')

In [ ]:
from sklearn.metrics import f1_score


def f1_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return f1_score(y, preds, average="micro")

In [ ]:
cross_val_score(pipeline_gaussiano, X_test, y_test, cv=5, scoring=f1_multietiqueta)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afirmo', 'agrego', 'algun', 'anadio', 'aseguro', 'comento', 'considero', 'dejo', 'demas', 'estabamos', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'explico', 'expreso', 'fueramos', 'fuesemos', 'habeis', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'indico', 'llego', 'manifesto', 'menciono', 'ningun', 'podra', 'podran', 'proximos', 'quedo', 'realizo', 'seais', 'senalo', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 'tendria', 't

array([0.65993266, 0.62794613, 0.62962963, 0.64814815, 0.61447811])

In [ ]:
pipeline_multinomial=make_pipeline(
    TfidfVectorizer(strip_accents='unicode', stop_words=stopwords_es, max_features=1000), #Dont check in all the words only in the 1000 most frequent
    DenseTransformer(),
    MultinomialNB()
)

In [ ]:
cross_val_score(pipeline_gaussiano, X_test, y_test, cv=5, scoring=f1_multietiqueta)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afirmo', 'agrego', 'algun', 'anadio', 'aseguro', 'comento', 'considero', 'dejo', 'demas', 'estabamos', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'explico', 'expreso', 'fueramos', 'fuesemos', 'habeis', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'indico', 'llego', 'manifesto', 'menciono', 'ningun', 'podra', 'podran', 'proximos', 'quedo', 'realizo', 'seais', 'senalo', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 'tendria', 't

array([0.65993266, 0.62794613, 0.62962963, 0.64814815, 0.61447811])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizador_count = CountVectorizer(strip_accents='unicode', stop_words=stopwords_es, binary=True, max_features=1000)

In [ ]:
vectorizador_count.fit(noticias.descripcion)
vectorizador_count.vocabulary_

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afirmo', 'agrego', 'algun', 'anadio', 'aseguro', 'comento', 'considero', 'dejo', 'demas', 'estabamos', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'explico', 'expreso', 'fueramos', 'fuesemos', 'habeis', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'indico', 'llego', 'manifesto', 'menciono', 'ningun', 'podra', 'podran', 'proximos', 'quedo', 'realizo', 'seais', 'senalo', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 'tendria', 't

{'deberia': np.int64(251),
 'historia': np.int64(451),
 'real': np.int64(807),
 'policia': np.int64(739),
 'barcelona': np.int64(111),
 'espana': np.int64(343),
 'resulta': np.int64(830),
 'caso': np.int64(155),
 'cuerpo': np.int64(240),
 'situacion': np.int64(879),
 'arte': np.int64(88),
 'anos': np.int64(74),
 'ayuda': np.int64(106),
 'nino': np.int64(640),
 'zona': np.int64(998),
 'ciudad': np.int64(171),
 'comportamiento': np.int64(192),
 'programa': np.int64(775),
 'llevan': np.int64(543),
 'cabo': np.int64(125),
 'lugares': np.int64(553),
 'territorio': np.int64(916),
 'espacio': np.int64(342),
 've': np.int64(956),
 'rio': np.int64(836),
 'razon': np.int64(805),
 'actividad': np.int64(46),
 'cosa': np.int64(228),
 'consiste': np.int64(212),
 'partido': np.int64(699),
 'texto': np.int64(918),
 'lineas': np.int64(530),
 'conocido': np.int64(206),
 'vio': np.int64(976),
 'musica': np.int64(625),
 'new': np.int64(638),
 'dejo': np.int64(261),
 'imagenes': np.int64(468),
 'pluton': n

In [ ]:
pipeline_bernoulli = make_pipeline(
    vectorizador_count,
    DenseTransformer(),
    BernoulliNB()
)

In [ ]:
pipeline_bernoulli.fit(X=X_train, y=y_train)


Pipeline(steps=[('countvectorizer',
                 CountVectorizer(binary=True, max_features=1000,
                                 stop_words=['0', '1', '2', '3', '4', '5', '6',
                                             '7', '8', '9', '_', 'a',
                                             'actualmente', 'acuerdo',
                                             'adelante', 'ademas', 'además',
                                             'adrede', 'afirmó', 'agregó',
                                             'ahi', 'ahora', 'ahí', 'al',
                                             'algo', 'alguna', 'algunas',
                                             'alguno', 'algunos', 'algún', ...],
                                 strip_accents='unicode')),
                ('densetransformer', DenseTransformer()),
                ('bernoullinb', BernoulliNB())])

In [ ]:
cross_val_score(pipeline_bernoulli, X_test, y_test, cv=5, scoring=f1_multietiqueta)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afirmo', 'agrego', 'algun', 'anadio', 'aseguro', 'comento', 'considero', 'dejo', 'demas', 'estabamos', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'explico', 'expreso', 'fueramos', 'fuesemos', 'habeis', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'indico', 'llego', 'manifesto', 'menciono', 'ningun', 'podra', 'podran', 'proximos', 'quedo', 'realizo', 'seais', 'senalo', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 'tendria', 't

array([0.65151515, 0.67676768, 0.64141414, 0.64814815, 0.64141414])

In [ ]:
# Create an instance of DenseTransformer
dense_transformer_instance = DenseTransformer()

# Call fit_transform on the instance with the vectorized data
hola=dense_transformer_instance.fit_transform(TfidfVectorizer(strip_accents='unicode', stop_words=stopwords_es, max_features=1000).fit_transform(X_train))

# Call an instance od MultinomialNB
multinomial_nb_instance = MultinomialNB()

#Call fit on the instance with the y train
multinomial_nb_instance.fit(hola, y_train)

cross_val_score(multinomial_nb_instance, X=hola,y= y_train, cv=3, scoring=f1_multietiqueta)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afirmo', 'agrego', 'algun', 'anadio', 'aseguro', 'comento', 'considero', 'dejo', 'demas', 'estabamos', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'explico', 'expreso', 'fueramos', 'fuesemos', 'habeis', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'indico', 'llego', 'manifesto', 'menciono', 'ningun', 'podra', 'podran', 'proximos', 'quedo', 'realizo', 'seais', 'senalo', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 'tendria', 't

array([0.67994803, 0.68514508, 0.69423993])